In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords


In [2]:
#loading the data

data=pd.read_csv('data/imdb_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [3]:
data.head()


,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [5]:
data['sentiment'].value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [6]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("movie is not good")

'movie is not good'

In [7]:
train_data = data[:int(0.7*len(data))]
test_and_dev_data = data[int(0.7*len(data)):]


dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())

print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     523 non-null    object
 1   sentiment  523 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.3+ KB
None



In [8]:
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 635 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     113 non-null    object
 1   sentiment  113 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB
None


In [9]:
print(dev_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 523 to 634
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     112 non-null    object
 1   sentiment  112 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB
None


In [10]:
train_data['sentiment'].value_counts()

0    292
1    231
Name: sentiment, dtype: int64

In [11]:
test_data['sentiment'].value_counts()

1    72
0    41
Name: sentiment, dtype: int64

In [12]:
dev_data['sentiment'].value_counts()

1    83
0    29
Name: sentiment, dtype: int64

In [13]:
X_train = train_data['review']
y_train = train_data['sentiment']
# print(X_train)
# print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
# print(X_dev)
# print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
# print(X_test)
# print(y_test)

In [14]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.5583173996175909
Positive review class' prior probability-- 0.4416826003824092


In [15]:
def build_word_dict(data):
    word_freq = {}
    all_words=[]
    omit=[]
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                    all_words.append(word)
                else:
                    word_freq[word] += 1
    for word in word_freq:
        if word_freq[word] <5:
            omit.append(word)

    for word in omit:
        word_freq.pop(word,None)
        
            

    return word_freq, all_words

    
       
allwords_freq,allwords_train=build_word_dict(X_train)
print("Total words--",len(allwords_train))


train_pos_reviews=train_data.loc[train_data.sentiment==1].review
pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
print("Positive words--",len(pos_words))


train_neg_reviews=train_data.loc[train_data.sentiment==0].review
neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
print("Negative words--",len(neg_words))

Total words-- 2482
Positive words-- 1267
Negative words-- 1748


In [16]:
allwords_freq.pop('', None)

for word in allwords_freq:
    print(word,"--",allwords_freq[word],"    probability=",allwords_freq[word]/len(X_train))

a -- 194     probability= 0.37093690248565964
very -- 35     probability= 0.06692160611854685
about -- 24     probability= 0.045889101338432124
movie -- 92     probability= 0.17590822179732313
man -- 8     probability= 0.015296367112810707
or -- 24     probability= 0.045889101338432124
not -- 36     probability= 0.06883365200764818
characters -- 19     probability= 0.036328871892925434
of -- 160     probability= 0.30592734225621415
who -- 26     probability= 0.0497131931166348
out -- 31     probability= 0.05927342256214149
half -- 6     probability= 0.011472275334608031
more -- 22     probability= 0.04206500956022945
was -- 98     probability= 0.18738049713193117
the -- 309     probability= 0.5908221797323135
with -- 50     probability= 0.09560229445506692
disappointed -- 5     probability= 0.009560229445506692
as -- 47     probability= 0.08986615678776291
poor -- 6     probability= 0.011472275334608031
acting -- 22     probability= 0.04206500956022945
lines -- 6     probability= 0.011

In [17]:
neg_words_freq.pop('', None)
neg_condi_prob={}
for word in neg_words_freq:
    print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
    neg_condi_prob[word]=neg_words_freq[word]/len(train_neg_reviews)

a -- 93       Conditional probability of a  given sentiment is neg= 0.3184931506849315
very -- 21       Conditional probability of very  given sentiment is neg= 0.07191780821917808
about -- 16       Conditional probability of about  given sentiment is neg= 0.0547945205479452
movie -- 52       Conditional probability of movie  given sentiment is neg= 0.1780821917808219
or -- 18       Conditional probability of or  given sentiment is neg= 0.06164383561643835
not -- 30       Conditional probability of not  given sentiment is neg= 0.10273972602739725
characters -- 10       Conditional probability of characters  given sentiment is neg= 0.03424657534246575
of -- 82       Conditional probability of of  given sentiment is neg= 0.2808219178082192
who -- 15       Conditional probability of who  given sentiment is neg= 0.05136986301369863
out -- 14       Conditional probability of out  given sentiment is neg= 0.04794520547945205
half -- 5       Conditional probability of half  given sentiment is 

In [18]:
neg_condi_prob

{'a': 0.3184931506849315,
 'very': 0.07191780821917808,
 'about': 0.0547945205479452,
 'movie': 0.1780821917808219,
 'or': 0.06164383561643835,
 'not': 0.10273972602739725,
 'characters': 0.03424657534246575,
 'of': 0.2808219178082192,
 'who': 0.05136986301369863,
 'out': 0.04794520547945205,
 'half': 0.017123287671232876,
 'more': 0.030821917808219176,
 'was': 0.2089041095890411,
 'the': 0.5856164383561644,
 'with': 0.09931506849315068,
 'as': 0.08904109589041095,
 'poor': 0.02054794520547945,
 'acting': 0.0547945205479452,
 'lines': 0.02054794520547945,
 'almost': 0.02054794520547945,
 'ridiculous': 0.017123287671232876,
 'even': 0.07876712328767123,
 'and': 0.3047945205479452,
 'plot': 0.0547945205479452,
 'little': 0.03767123287671233,
 'to': 0.23972602739726026,
 'because': 0.0410958904109589,
 'it': 0.24315068493150685,
 'if': 0.05136986301369863,
 'its': 0.07191780821917808,
 'i': 0.22602739726027396,
 'wasted': 0.017123287671232876,
 'predictable': 0.02054794520547945,
 'had': 

In [19]:
pos_words_freq.pop('', None)
pos_condi_prob={}
for word in pos_words_freq:
    print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",pos_words_freq[word]/len(train_pos_reviews))
    pos_condi_prob[word]=pos_words_freq[word]/len(train_pos_reviews)

a -- 101       Conditional probability of a  given sentiment is neg= 0.43722943722943725
the -- 138       Conditional probability of the  given sentiment is neg= 0.5974025974025974
when -- 7       Conditional probability of when  given sentiment is neg= 0.030303030303030304
to -- 56       Conditional probability of to  given sentiment is neg= 0.24242424242424243
best -- 10       Conditional probability of best  given sentiment is neg= 0.04329004329004329
is -- 66       Conditional probability of is  given sentiment is neg= 0.2857142857142857
was -- 37       Conditional probability of was  given sentiment is neg= 0.16017316017316016
that -- 30       Conditional probability of that  given sentiment is neg= 0.12987012987012986
in -- 45       Conditional probability of in  given sentiment is neg= 0.19480519480519481
movie -- 40       Conditional probability of movie  given sentiment is neg= 0.17316017316017315
his -- 10       Conditional probability of his  given sentiment is neg= 0.043290

In [20]:
pos_condi_prob

{'a': 0.43722943722943725,
 'the': 0.5974025974025974,
 'when': 0.030303030303030304,
 'to': 0.24242424242424243,
 'best': 0.04329004329004329,
 'is': 0.2857142857142857,
 'was': 0.16017316017316016,
 'that': 0.12987012987012986,
 'in': 0.19480519480519481,
 'movie': 0.17316017316017315,
 'his': 0.04329004329004329,
 'for': 0.1038961038961039,
 'it': 0.24242424242424243,
 'and': 0.45454545454545453,
 'good': 0.06926406926406926,
 'as': 0.09090909090909091,
 'of': 0.33766233766233766,
 'were': 0.047619047619047616,
 'very': 0.06060606060606061,
 'at': 0.05627705627705628,
 'look': 0.030303030303030304,
 'its': 0.09956709956709957,
 'made': 0.030303030303030304,
 'so': 0.0735930735930736,
 'this': 0.2857142857142857,
 'on': 0.05627705627705628,
 'be': 0.06060606060606061,
 'ever': 0.025974025974025976,
 'film': 0.15584415584415584,
 'i': 0.27705627705627706,
 'other': 0.03896103896103896,
 'acting': 0.025974025974025976,
 'or': 0.025974025974025976,
 'cinematography': 0.02164502164502164

In [21]:
def score(real, prediction):
    count = 0
    for i in range(len(real)):
        if real[i] == prediction[i]:
            count+=1
    accuracy=count/len(real)*100
    return accuracy

In [22]:
def pred(data,ans):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev))
print("Accuracy on test data--",pred(X_test,y_test))

Accuracy on training data-- 57.93499043977055
Accuracy on development data-- 25.892857142857146
Accuracy on test data-- 36.283185840707965
